In [18]:
import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import pickle
import sys

In [19]:
def findPath(relative_path):
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    target_path = os.path.join(parent_dir, relative_path)
    return target_path


In [20]:
#importing the sentiment analyzer

sys.path.insert(0, findPath('sentiment_analysis')) # Add the folder to the system path

from SentimentAnalyzer import SentimentAnalyzer

analyzing sentiment of each comment, and storing data

In [21]:
comment_dict = {}

relative_path = os.path.join('reddit', '2024')
comments_files = os.listdir(findPath(relative_path))

for filename in comments_files:
    path = os.path.join(relative_path, filename)
    filepath = findPath(path)
    game_id = int(filename.replace(".csv", ""))
    comment_dict[game_id] = []

    with open(filepath, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)    # skip the header row
        
        for row in reader:
            timestamp = row[0]
            comment = row[1]
            upvotes = row[2]

            # Analyze sentiment of the comment
            sentiment = SentimentAnalyzer.sentimentScore(comment)
            comment_dict[game_id].append([timestamp, upvotes, sentiment])

print(len(comment_dict))
print(comment_dict[1])

74
[['2024-03-22T13:02:43+00:00', '7', -0.4588], ['2024-03-22T13:02:53+00:00', '5', 0.0], ['2024-03-22T13:03:27+00:00', '6', 0.0], ['2024-03-22T13:05:43+00:00', '3', 0.0], ['2024-03-22T13:06:51+00:00', '4', 0.0], ['2024-03-22T13:07:03+00:00', '2', 0.0], ['2024-03-22T13:07:33+00:00', '2', 0.0], ['2024-03-22T13:07:54+00:00', '4', 0.2225], ['2024-03-22T13:08:30+00:00', '5', 0.2732], ['2024-03-22T13:08:34+00:00', '2', -0.3182], ['2024-03-22T13:08:37+00:00', '2', 0.0], ['2024-03-22T13:09:28+00:00', '-1', 0.7935], ['2024-03-22T13:09:30+00:00', '3', -0.296], ['2024-03-22T13:09:38+00:00', '1', -0.34], ['2024-03-22T13:09:55+00:00', '5', 0.0], ['2024-03-22T13:10:04+00:00', '3', 0.0], ['2024-03-22T13:10:54+00:00', '2', -0.296], ['2024-03-22T13:11:05+00:00', '3', 0.0], ['2024-03-22T13:11:11+00:00', '3', 0.0], ['2024-03-22T13:11:23+00:00', '5', 0.0], ['2024-03-22T13:11:26+00:00', '3', 0.0], ['2024-03-22T13:11:33+00:00', '3', -0.34], ['2024-03-22T13:11:42+00:00', '8', 0.34], ['2024-03-22T13:11:44+00

sectioning game threads for each match into the timestamp buckets

In [22]:
with open('timestamps.pkl', 'rb') as file:
    timestamps = pickle.load(file)

print(timestamps)

{1: ['2024-03-22T14:25:30Z', '2024-03-22T14:30:13Z', '2024-03-22T14:34:49Z', '2024-03-22T14:40:19Z', '2024-03-22T14:44:54Z', '2024-03-22T14:50:28Z', '2024-03-22T14:55:08Z', '2024-03-22T15:00:36Z', '2024-03-22T15:05:28Z', '2024-03-22T15:09:18Z', '2024-03-22T15:14:50Z', '2024-03-22T15:19:39Z', '2024-03-22T15:25:24Z', '2024-03-22T15:30:12Z', '2024-03-22T15:35:28Z', '2024-03-22T15:39:42Z', '2024-03-22T15:45:18Z', '2024-03-22T15:49:25Z', '2024-03-22T15:53:44Z', '2024-03-22T15:59:57Z', '2024-03-22T16:05:34Z', '2024-03-22T16:09:57Z', '2024-03-22T16:15:28Z', '2024-03-22T16:19:51Z', '2024-03-22T16:24:49Z', '2024-03-22T16:29:56Z', '2024-03-22T16:35:07Z', '2024-03-22T16:40:34Z', '2024-03-22T16:45:01Z', '2024-03-22T16:49:32Z', '2024-03-22T16:55:16Z', '2024-03-22T16:59:41Z', '2024-03-22T17:02:42Z', '2024-03-22T17:09:21Z', '2024-03-22T17:14:49Z', '2024-03-22T17:19:59Z', '2024-03-22T17:25:35Z', '2024-03-22T17:30:32Z', '2024-03-22T17:34:04Z', '2024-03-22T17:40:06Z', '2024-03-22T17:43:55Z', '2024-03-22

In [26]:
comment_buckets = {}
for key, value in comment_dict.items():     # key is match number; value is the array of (data associated with comments)
    pos_timestamp = 0
    comment_buckets[key] = {}
    
    try:
        match_timestamps_list = timestamps[key]
    except Exception:
        print(f"Exception: Error in processing match {key}: match data not found")
        continue

    timestamp = match_timestamps_list[pos_timestamp]

    for comment in value:
        comment_timestamp = comment[0]
        
        if (comment_timestamp >= timestamp):    # go to next timestamp
            pos_timestamp += 1
            if pos_timestamp == len(timestamps[key]):   # go to next match (key)
                break
            timestamp = match_timestamps_list[pos_timestamp]

        # if timestamp not in comment_buckets[key]:
        #     comment_buckets[key][timestamp] = []

        comment_buckets[key].setdefault(timestamp, []).append(comment)
        # comment_buckets[key][timestamp] = comment_buckets[key][timestamp].append(comment)

        
print(comment_buckets[1])      

Exception: Error in processing match 10: match data not found
Exception: Error in processing match 52: match data not found
Exception: Error in processing match 62: match data not found
Exception: Error in processing match 63: match data not found
Exception: Error in processing match 66: match data not found
Exception: Error in processing match 70: match data not found
{'2024-03-22T14:25:30Z': [['2024-03-22T13:02:43+00:00', '7', -0.4588], ['2024-03-22T13:02:53+00:00', '5', 0.0], ['2024-03-22T13:03:27+00:00', '6', 0.0], ['2024-03-22T13:05:43+00:00', '3', 0.0], ['2024-03-22T13:06:51+00:00', '4', 0.0], ['2024-03-22T13:07:03+00:00', '2', 0.0], ['2024-03-22T13:07:33+00:00', '2', 0.0], ['2024-03-22T13:07:54+00:00', '4', 0.2225], ['2024-03-22T13:08:30+00:00', '5', 0.2732], ['2024-03-22T13:08:34+00:00', '2', -0.3182], ['2024-03-22T13:08:37+00:00', '2', 0.0], ['2024-03-22T13:09:28+00:00', '-1', 0.7935], ['2024-03-22T13:09:30+00:00', '3', -0.296], ['2024-03-22T13:09:38+00:00', '1', -0.34], ['202

In [27]:
# Pickle the dictionary
with open('comment_buckets.pkl', 'wb') as file:
    pickle.dump(comment_buckets, file)